In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic, great_circle
import lightgbm as lgb

from sklearn.metrics import mean_squared_error, mean_squared_log_error
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn import preprocessing

from datetime import date
from routingpy import Graphhopper
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df_train_full = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')

In [ ]:
client = Graphhopper(api_key='d6a565a3-71c5-4dbf-8d24-7608c1cf39d0')

In [ ]:
for df_ in [df_train_full, df_test]:
    try:
        df_['dur_osrm'] =[client.directions(locations=[[df_['Pickup Long'].iloc[i], df_['Pickup Lat'].iloc[i]], [df_['Destination Long'].iloc[i], df_['Destination Lat'].iloc[i]]], profile='scooter').duration for i in range(len(df_))]
    except:
        print(i)

In [ ]:
for df_ in [df_train_full, df_test]:
    df_['dist_geo'] =[geodesic((df_['Pickup Lat'].iloc[i], df_['Pickup Long'].iloc[i]), (df_['Destination Lat'].iloc[i], df_['Destination Long'].iloc[i])).km for i in range(len(df_))]
    df_['dist_cir'] =[great_circle((df_['Pickup Lat'].iloc[i], df_['Pickup Long'].iloc[i]), (df_['Destination Lat'].iloc[i], df_['Destination Long'].iloc[i])).km for i in range(len(df_))]

In [ ]:
for df_ in [df_train_full, df_test]:
    df_['time_place'] = df_['Placement - Time'].str[0:2].str.replace(':', '').astype(int)
    df_['time_confirm'] = df_['Confirmation - Time'].str[0:2].str.replace(':', '').astype(int)
    df_['time_arr'] = df_['Arrival at Pickup - Time'].str[0:2].str.replace(':', '').astype(int)
    df_['time_pickup'] = df_['Pickup - Time'].str[0:2].str.replace(':', '').astype(int)

In [ ]:
for df_ in [df_train_full, df_test]:
    df_['diff_place'] = df_['Confirmation - Day of Month'] - df_['Placement - Day of Month']
    df_['diff_arr'] = df_['Placement - Day of Month'] - df_['Arrival at Pickup - Day of Month']
    df_['diff_pickup'] = df_['Arrival at Pickup - Day of Month'] - df_['Pickup - Day of Month']

In [ ]:
le = preprocessing.LabelEncoder()
for feat in ['Vehicle Type', 'Platform Type','Personal or Business']:
    le_fitted = le.fit(df_train_full[feat])
    df_train_full[feat] = le_fitted.transform(df_train_full[feat])
    df_test[feat] = le_fitted.transform(df_test[feat])

In [ ]:
list_unpop = list(df_train_full['Rider Id'].value_counts()[df_train_full['Rider Id'].value_counts() < 40].index)
df_train_full['Rider Id'][df_train_full['Rider Id'].isin(list_unpop)] = '0'
df_test['Rider Id'][df_test['Rider Id'].isin(list_unpop)] = '0'

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df_train_full['Rider Id'].values.reshape(-1, 1))



categories_1_train = pd.DataFrame(enc.transform(df_train_full['Rider Id'].values.reshape(-1, 1)).toarray(), columns = ['rider_' + str(i) for i in range (0, len(df_train_full['Rider Id'].unique()))])
categories_1_test = pd.DataFrame(enc.transform(df_test['Rider Id'].values.reshape(-1, 1)).toarray(), columns = ['rider_' + str(i) for i in range (0, len(df_train_full['Rider Id'].unique()))])
df_train_full = pd.concat([df_train_full, categories_1_train], axis = 1)
df_test = pd.concat([df_test, categories_1_test], axis = 1)

In [ ]:
df_train_full.corr()['Time from Pickup to Arrival'].sort_values()

In [ ]:
df_train_full.to_csv('df_train_full_Sendy.csv')
df_test.to_csv('df_test_Sendy.csv')

In [ ]:
df_train_full = pd.read_csv('df_train_full_Sendy.csv')
df_test = pd.read_csv('df_test_Sendy.csv')

In [ ]:
for df_ in [df_train_full, df_test]:
    df_['diff_lats'] = df_['Pickup Lat'] - df_['Destination Lat']
    df_['diff_lons'] = df_['Pickup Long'] - df_['Destination Long']

In [ ]:
df_test.columns[:50]

In [ ]:
df_train_full['User Id'].value_counts()

In [ ]:
list_unpop = list(df_train_full['User Id'].value_counts()[df_train_full['User Id'].value_counts() < 100].index)
df_train_full['User Id'][df_train_full['User Id'].isin(list_unpop)] = '0'
df_test['User Id'][df_test['User Id'].isin(list_unpop)] = '0'

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df_train_full['User Id'].values.reshape(-1, 1))



categories_1_train = pd.DataFrame(enc.transform(df_train_full['User Id'].values.reshape(-1, 1)).toarray(), columns = ['User_' + str(i) for i in range (0, len(df_train_full['User Id'].unique()))])
categories_1_test = pd.DataFrame(enc.transform(df_test['User Id'].values.reshape(-1, 1)).toarray(), columns = ['User_' + str(i) for i in range (0, len(df_train_full['User Id'].unique()))])
df_train_full = pd.concat([df_train_full, categories_1_train], axis = 1)
df_test = pd.concat([df_test, categories_1_test], axis = 1)

In [ ]:
df_train_full.to_csv('df_train_full_Sendy.csv')
df_test.to_csv('df_test_Sendy.csv')